In [2]:
import numpy as np
import os

DATASETPATH = "/home/zantyr/.dataset"
MODEL_ROOT = "./models"

train_source = np.load(os.path.join(DATASETPATH, "train_source.bin.npy"))
valid_source = np.load(os.path.join(DATASETPATH, "valid_source.bin.npy"))
train_target = np.load(os.path.join(DATASETPATH, "train_target.bin.npy"))
valid_target = np.load(os.path.join(DATASETPATH, "valid_target.bin.npy"))

# set this to "-1" to disable GPU, set this to other numbers in multi-GPU systems
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [3]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Lambda, LeakyReLU, TimeDistributed, Flatten, BatchNormalization
import keras.backend as K

MAX = train_source.max()

def mk_model():
    wrap = lambda x: BatchNormalization()(LeakyReLU(0.01)(x))
    nw_input = lyr = Input((1248, 129))
    lyr = Lambda(lambda x: x / MAX)(lyr)
    lyr = Lambda(K.expand_dims)(lyr)
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = Lambda(lambda x:K.permute_dimensions(x, (0,1,3,2)))(lyr)
    lyr = wrap(Dense(96)(lyr))
    lyr = wrap(Dense(64)(lyr))
    lyr = TimeDistributed(Flatten())(lyr)
    lyr = Dense(257, use_bias=False)(lyr)
    lyr = Lambda(lambda x: x * MAX)(lyr)
    model = Model(nw_input, lyr)
    return model

import keras.callbacks as kc

mk_model().summary()

def training(model):
    for lr in [3e-4, 1e-4, 3e-5, 1e-5, 3e-6, 1e-6]:
        model.compile(keras.optimizers.Adam(lr), 'mse')
        model.fit(train_source, train_target, validation_data=(valid_source, valid_target), batch_size=8, epochs=100, callbacks=[kc.TensorBoard("./tf_logs"), kc.EarlyStopping()])

Using TensorFlow backend.


Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
Instructions for updating:
The TensorFlow Distributions library has moved to TensorFlow Probability (https://github.com/tensorflow/probability). You should update all references to use `tfp.distributions` instead of `tf.distributions`.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 1248, 129)         0         
_________________________________________________________________
lambda_1 (Lambda)            (None, 1248, 129)         0         
_________________________________________________________________
lambda_2 (Lambda)            (None, 1248, 129, 1)      0         
_________________________________________________________________


In [4]:
import datetime

def experiment(mk_model):
    model = mk_model()
    name = datetime.datetime.now().isoformat() + ".h5"
    try:
        training(model)
    finally:
        keras.models.save_model(model, os.path.join(MODEL_ROOT, name))
        print("Model path: ", os.path.join(MODEL_ROOT, name))
    return model
        
experiment(mk_model)

Train on 160 samples, validate on 20 samples
Epoch 1/100
160/160 [==============================] - 4s 24ms/step - loss: 1058.8642 - val_loss: 933.0909
Epoch 2/100
160/160 [==============================] - 2s 14ms/step - loss: 581.8323 - val_loss: 775.0587
Epoch 3/100
160/160 [==============================] - 2s 14ms/step - loss: 280.9676 - val_loss: 734.1549
Epoch 4/100
160/160 [==============================] - 2s 14ms/step - loss: 165.9070 - val_loss: 581.3661
Epoch 5/100
160/160 [==============================] - 2s 14ms/step - loss: 101.4906 - val_loss: 369.1296
Epoch 6/100
160/160 [==============================] - 2s 14ms/step - loss: 70.7532 - val_loss: 242.3070
Epoch 7/100
160/160 [==============================] - 2s 15ms/step - loss: 56.9242 - val_loss: 170.1039
Epoch 8/100
160/160 [==============================] - 2s 14ms/step - loss: 46.0996 - val_loss: 132.2921
Epoch 9/100
160/160 [==============================] - 2s 14ms/step - loss: 41.5238 - val_loss: 92.7083
Epoch

In [11]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Conv2D, Lambda, LeakyReLU, TimeDistributed, Flatten, BatchNormalization
import keras.backend as K

MAX = train_source.max()

def mk_model():
    wrap = lambda x: BatchNormalization()(LeakyReLU(0.01)(x))
    nw_input = lyr = Input((1248, 129))
    lyr = Lambda(lambda x: x / MAX)(lyr)
    lyr = Lambda(K.expand_dims)(lyr)
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [5,1], padding="same", activation=None)(lyr))
    lyr = wrap(Conv2D(8, [1,5], padding="same", activation=None)(lyr))
    lyr = Lambda(lambda x:K.permute_dimensions(x, (0,1,3,2)))(lyr)
    lyr = wrap(Dense(96)(lyr))
    lyr = wrap(Dense(64)(lyr))
    lyr = TimeDistributed(Flatten())(lyr)
    lyr = Dense(257, use_bias=False)(lyr)
    lyr = Lambda(lambda x: x * MAX)(lyr)
    model = Model(nw_input, lyr)
    return model

import keras.callbacks as kc

mk_model().summary()

def training(model):
    for lr in [3e-4, 1e-4, 3e-5, 1e-5, 3e-6, 1e-6]:
        model.compile(keras.optimizers.Adam(lr), 'mse')
        model.fit(train_source, train_target, validation_data=(valid_source, valid_target), batch_size=8, epochs=100, callbacks=[kc.TensorBoard("./tf_logs"), kc.EarlyStopping()])

In [5]:
MAX

26.937874